In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import math

In [2]:
## length of the robot arms 
l_1 = 2
l_2 = 2
l_3 = 2


In [3]:
## joint constraints
q1_bound = (0, math.pi)
q2_bound = (-math.pi, 0)
q3_bound = (-math.pi, math.pi)

In [4]:
## End effector coordinates in cartesian space
## q1, q2, and q3 are joint angles

## Forward kinematic model
## xe is the end effector x coordinate
def x_e(q1, q2, q3):
    return l_1 * np.cos(q1) + l_2 * np.cos(q1+q2) + l_3 * np.cos(q1+q2+q3)

## xe is the end effector y coordinate
def y_e(q1, q2, q3):
    return l_1 * np.sin(q1) + l_2 * np.sin(q1+q2) + l_3 * np.sin(q1+q2+q3)

## end effector orientation
def orn_e(q1, q2, q3):
    return q1 + q2 + q3

In [5]:
## gerenate random joint angles and their corresponding end effector cartesian positions'
def gen_data(dataset_size):
    ## sample random q1, q2 and q3 (joint angles)
    q1_train = np.random.uniform(low=q1_bound[0], high=q1_bound[1], size=(dataset_size, 1))
    q2_train = np.random.uniform(low=q2_bound[0], high=q2_bound[1], size=(dataset_size, 1))
    q3_train = np.random.uniform(low=q3_bound[0], high=q3_bound[1], size=(dataset_size, 1))

    ## generate random data
    xe_train = x_e(q1_train, q2_train, q3_train)
    ye_train = y_e(q1_train, q2_train, q3_train)
    orn_train = orn_e(q1_train, q2_train, q3_train)

    targets = np.concatenate((q1_train, q2_train, q3_train), axis=1) ## targets as joint positions
    inputs = np.concatenate((xe_train, ye_train, orn_train), axis=1) ## inputs as end effector cartesian coordinates

    return inputs, targets

In [6]:
train_inputs, train_targets = gen_data(1000) ## training data for training the neural network
test_inputs, test_targets = gen_data(100) ## test data for validating the training performances

In [7]:
## Neural network is your IK model
model = nn.Sequential(
    nn.Linear(3, 100), ## input layer take cartesian coordinates
    nn.Linear(100, 3) ## output layer predicts corresponding joint positions
)

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [9]:
def val_model(net, t_inputs, t_targets):
    net.eval() ## set the model to evalution
    loss_list = list()
    for t_inp, t_tgt in zip(t_inputs, t_targets): ## feed the test data iteratively to the model
        t_inp = torch.tensor(t_inp, dtype=torch.float32) ## make the input tensor
        t_tgt = torch.tensor(t_tgt) ## make the target tensor
        out = net(t_inp) ## call the trained model
        loss = nn.MSELoss()(out, t_tgt)
        loss_list.append(loss.item())
    return np.mean(np.array(loss_list))

In [ ]:
for e in range(1000):
    epoch_train_loss = list()
    for inp, tgt in zip(train_inputs, train_targets):
        inp = torch.tensor(inp, dtype=torch.float32)
        tgt = torch.tensor(tgt)
        optimizer.zero_grad()
        out = model(inp)
        loss = nn.MSELoss()(out.double(), tgt)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())

    ## validate
    with torch.no_grad():
        val_error = val_model(model, test_inputs, test_targets)

    print("Train Error: {}, val Error: {}".format(np.mean(np.array(epoch_train_loss)), val_error))

    